In [1]:
import tensorflow as tf
import os

In [2]:
dataset_url ="dataset/PetImages"
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join(dataset_url, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            img_bytes = tf.io.read_file(fpath)
            tf.io.decode_image(img_bytes)
        except Exception as e:
            print(f"Bad file found, deleting: {fpath}")
            num_skipped += 1
            os.remove(fpath)

Bad file found, deleting: dataset/PetImages\Cat\10404.jpg
Bad file found, deleting: dataset/PetImages\Cat\4351.jpg
Bad file found, deleting: dataset/PetImages\Cat\666.jpg
Bad file found, deleting: dataset/PetImages\Cat\Thumbs.db
Bad file found, deleting: dataset/PetImages\Dog\11233.jpg
Bad file found, deleting: dataset/PetImages\Dog\11702.jpg
Bad file found, deleting: dataset/PetImages\Dog\11912.jpg
Bad file found, deleting: dataset/PetImages\Dog\2317.jpg
Bad file found, deleting: dataset/PetImages\Dog\2494.jpg
Bad file found, deleting: dataset/PetImages\Dog\9500.jpg
Bad file found, deleting: dataset/PetImages\Dog\Thumbs.db


In [ ]:
image_size=(256,256)
batch_size = 32

In [ ]:
train_dr = tf.keras.utils.image_dataset_from_directory(
    dataset_url,
    subset="Training",
    seed=123,
    validation_split=0.2,
    image_size = image_size,
    batch = batch_size

)

In [ ]:
validation_dr = tf.keras.utils.image_dataset_from_directory(
    dataset_url,
    subset="Validation",
    seed=123,
    validation_split=0.2,
    image_size = image_size,
    batch = batch_size

)

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(input=(256,256,3)),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=256, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=128, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(units=1, activation="sigmoid")

    ]
)